# CSC337: Coursework 1

## Task 1: Design 5
### ?
<b>Visual Design Type:</b> ?

<b>Name of Tool:</b> Altair

<b>Country:</b> ?

<b>Year:</b> ?

<b>Visual Mappings:</b>
+ ?

Additionally, the following points were deliberated:
+ ?

In [1]:
import pandas as pd
import altair as alt
import datetime as dt

Debug options

In [2]:
# font to use for chart labels
__CHART_FONT__ = 'Circular'

# DEBUG: disable maximum row prevention (cripples chart performance)
alt.data_transformers.disable_max_rows()

# DEBUG: set max rows/columns in pandas table previewer
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

Load the GPPD data set and sequentially derive the required information

In [3]:
# load GPPD data set into pandas array
d_plants = pd.read_csv('../data/global_power_plant_database.csv')

# keep only necessary columns
d_splom = d_plants[['country_long', 'name', 'capacity_mw', 'latitude', 'primary_fuel']]

# calculate age of plants
d_splom['age'] = dt.datetime.now().year - d_plants['commissioning_year']

# combine national gen figures with US EPA figures for more complete figures
d_splom['est_gen'] = d_plants['estimated_generation_gwh'].fillna(
    d_plants['generation_gwh_2014']
)

# count number of fuel types listed
d_splom['num_fuels'] = d_plants[['other_fuel1', 'other_fuel2', 'other_fuel3']].apply(lambda x: x.count(), axis=1) + 1

# truncate all figures for nicer presentation
d_splom = d_splom.round(0)

# remove records if null in est_gen or age
d_splom = d_splom.dropna(
    subset=['est_gen', 'age']
).reset_index(
    drop=True
)

# give variables human-friendly names here so less chart faffing
d_splom.columns = ['Country', 'Plant name', 'Capacity (MW)', 'Latitude', 'Main fuel type', 'Age', 'Estimated generation (GWh)', 'Number of fuels']

C:\Users\mhmat\Anaconda3\envs\CSC327\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mhmat\Anaconda3\envs\CSC327\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\mhmat\Anaconda3\envs\CSC327\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

Draw the visualisation

In [4]:
# create base chart shape
base = alt.Chart(
    d_splom,
    height=250,
    width=250
).mark_circle(
    opacity=0.5
).encode(
    color=alt.Color(
        'Main fuel type',
        scale=alt.Scale(
            scheme='tableau20'
        ),
        legend=alt.Legend(
            title='Main fuel type',
            titleFontSize=18,
            titleFont=__CHART_FONT__,
            labelFontSize=16,
            labelFont=__CHART_FONT__
        )
    ),
    tooltip=[
        alt.Tooltip('Plant name:N'),
        alt.Tooltip('Country:N'),
        alt.Tooltip('Main fuel type:N')
    ]
)

# create default axis wih base style
default_axis = alt.Axis(
    titleFontSize=18,
    titleFont=__CHART_FONT__,
    labelFontSize=14,
    labelFont=__CHART_FONT__,
    labelFlush=False
)

# iterate through each grid placement and make a chart
variables = ['Age', 'Capacity (MW)', 'Estimated generation (GWh)', 'Latitude', 'Number of fuels']
charts = []

for y in variables:
    for x in variables:
        if ((x == variables[0]) and (y == variables[-1])):
            charts.append(
                base.encode(
                    x=alt.X(x, axis=default_axis),
                    y=alt.Y(y, axis=default_axis)
                )
            )
        elif (x == variables[0]):
            charts.append(
                base.encode(
                    x=alt.X(x, axis=None),
                    y=alt.Y(y, axis=default_axis)
                )
            )
        elif (y == variables[-1]):
            charts.append(
                base.encode(
                    x=alt.X(x, axis=default_axis),
                    y=alt.Y(y, axis=None)
                )
            )
        else:
            charts.append(
                base.encode(
                    x=alt.X(x, axis=None),
                    y=alt.Y(y, axis=None)
                )
            )

# plot the charts in a nxn grid
alt.ConcatChart(
    concat=charts,
    columns=len(variables),
    padding=40,
    title=alt.TitleParams(
        text='Comparing Global Power Plant Characteristics',
        fontSize=28,
        font=__CHART_FONT__
    )
)

alt.ConcatChart(...)